In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
response = requests.get("https://www.lrt.lt/en/news-in-english")
doc = BeautifulSoup(response.content)

In [5]:
items = doc.select(".news")
len(items)

42

In [6]:
items[0]

<div class="news">
<div class="media-block">
<div class="media-block__container">
<div class="media-block__wrapper">
<img alt="Gitanas NausÄda, Ingrida Å imonytÄ, Arvydas AnuÅ¡auskas" class="media-block__image js-lazy lazy-load" data-src="/img/2023/02/18/1451044-637891-150x84.jpg" src="/images/pxl.gif" title="Gitanas NausÄda, Ingrida Å imonytÄ, Arvydas AnuÅ¡auskas"/>
</div>
<a aria-label="Lithuanian PM voices confidence in defence minister" class="media-block__link" href="/en/news-in-english/19/2050571/lithuanian-pm-voices-confidence-in-defence-minister" title="Lithuanian PM voices confidence in defence minister"></a>
</div>
</div>
<div class="news__content">
<div class="info-block">
<a class="info-block__link" href="/en/news-in-english" title="News">News</a>
<span class="info-block__time-before" title="2023.08.04 17:03"> <strong>4 h ago</strong>
</span>
</div>
<h3 class="news__title"><a href="/en/news-in-english/19/2050571/lithuanian-pm-voices-confidence-in-defence-minister" title

In [15]:
articles = []
for item in items:
    headline = item.select_one('h3').text
    url = item.select_one('a')['href']
    img = item.select_one('img').get('data-src', None)
    article = {
        'url': url,
        'headline': headline,
        'img_path': img
    }
    articles.append(article)
len(articles)

42

In [16]:
df=pd.DataFrame(articles)
df.head()

,url,headline,img_path
0,/en/news-in-english/19/2050571/lithuanian-pm-v...,Lithuanian PM voices confidence in defence min...,/img/2023/02/18/1451044-637891-150x84.jpg
1,/en/news-in-english/19/2050524/lithuania-deems...,"Lithuania deems 1,164 Belarusian and Russian n...",/img/2022/03/01/1207094-733403-150x84.jpg
2,/en/news-in-english/19/2050519/vilnius-ex-mayo...,Vilnius ex-mayor Å imaÅ¡ius returns to private...,/img/2023/04/17/1491828-404400-150x84.jpg
3,/en/news-in-english/19/2050432/lithuania-s-sup...,Lithuania's support to Ukraine includes helico...,/img/2023/07/20/1555673-67448-150x84.jpg
4,/en/news-in-english/19/2050380/latvia-to-ask-t...,Latvia to ask thousands of Russian citizens to...,/img/2019/08/05/485338-154587-150x84.jpg


## What to do with the results????

## Approach 1: Save it to a simple single csv file.

Just like https://github.com/jsoma/automatic-scraper-bbc

I want a CSV that always has the most current list of headlines in it, and maybe i want to be able to see a diff of the changes every time something is updated.

In [17]:
df.to_csv("headlines.csv", index=False)

### Approach 2: Save a different file every time we run the scraper

This is good for having easily-accessible data that is maybe weekly or monthly or even daily if you don't mind having hundreds of files in a single folder. Great for easily browsing to a current date and time (or just date) in your dataset.

In [18]:
import os

# Try to create a folder called 'data'
# and if it exists DON'T THROW AN ERROR
os.makedirs("data", exist_ok=True)

In [22]:
from datetime import datetime

date_string = datetime.now().strftime("%Y-%m-%d")
filepath = f"data/{date_string}.csv"

df.to_csv(filepath, index=False)

In [23]:
# Add a new column for today's date

# You could just use datetime.now() and get the entire datetime
# or another strftime but this is fine
df['scrape_date'] = datetime.now().strftime("%Y-%m-%d")
df.head()

,url,headline,img_path,scrape_date
0,/en/news-in-english/19/2050571/lithuanian-pm-v...,Lithuanian PM voices confidence in defence min...,/img/2023/02/18/1451044-637891-150x84.jpg,2023-08-04
1,/en/news-in-english/19/2050524/lithuania-deems...,"Lithuania deems 1,164 Belarusian and Russian n...",/img/2022/03/01/1207094-733403-150x84.jpg,2023-08-04
2,/en/news-in-english/19/2050519/vilnius-ex-mayo...,Vilnius ex-mayor Å imaÅ¡ius returns to private...,/img/2023/04/17/1491828-404400-150x84.jpg,2023-08-04
3,/en/news-in-english/19/2050432/lithuania-s-sup...,Lithuania's support to Ukraine includes helico...,/img/2023/07/20/1555673-67448-150x84.jpg,2023-08-04
4,/en/news-in-english/19/2050380/latvia-to-ask-t...,Latvia to ask thousands of Russian citizens to...,/img/2019/08/05/485338-154587-150x84.jpg,2023-08-04


In [24]:
# If it exists, open it
# If it doesn't, just make a blank dataframe
# could also use os.path.exists to check if the file exists
# but honestly try/except is the easiest route to go here
try:
    existing_df = pd.read_csv("always-updated.csv")
except:
    existing_df = pd.DataFrame([])
existing_df.head()

""
